In [13]:
from logs import logDecorator as lD 
import jsonref, pprint
from lib.databaseIO import pgIO

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
from ipywidgets import interact, interact_manual
import pickle


## 1. Load Data

In [ ]:
cgimeds = pickle.load(open('../data/intermediate/all_cgimeds.pkl','rb'))

In [14]:
full_data = pickle.load(open('../data/intermediate/full_dataset2.pkl','rb'))
patient_list = full_data.PatientID.unique()
print(f'There are {len(patient_list)} patients in the MDD cohort of interest.')
full_data.head(10)

There are 34895 patients in the MDD cohort of interest.


,PatientID,VisitID,VisitName,VisitType,Age,Days,Diagnosis,DSMNo,Medication,Dose,CGI
0,9,476,Group,Outpatient,20,1073,Alcohol Abuse,305.00,clorazepate,15.0,4
1,9,477,Group,Outpatient,20,1073,Alcohol Abuse,305.00,clorazepate,15.0,2
2,9,478,Group,Outpatient,19,1032,Alcohol Abuse,305.00,clorazepate,15.0,4
3,9,481,Progress Note,Outpatient,18,667,Alcohol Abuse,305.00,clorazepate,15.0,2
4,9,482,Progress Note,Outpatient,18,508,Alcohol Abuse,305.00,clorazepate,15.0,5
5,9,497,Group,Outpatient,18,436,Alcohol Abuse,305.00,clorazepate,15.0,4
6,9,502,Progress Note,Substance Evaluation,18,318,Alcohol Abuse,305.00,clorazepate,15.0,1
7,19,754,Evaluation,Discharge summary,39,542,Alcohol Intoxication,303.00,propranolol,25.0,4
8,19,755,Progress Note,Substance Evaluation,39,542,Nicotine Dependence,305.1,propranolol,25.0,4
9,19,756,Progress Note,Substance Evaluation,39,541,Nicotine Dependence,305.1,propranolol,25.0,4


In [21]:
def getPatientData(pid, data):
    patientData = data.loc[data.PatientID == pid].drop('PatientID', axis=1)
    if 'Days' in patientData.columns:
        patientData.sort_values('Days', inplace=True)
    return patientData

def getComparativePopulation(pid, data):
    cpData =  data.loc[data.PatientID != pid].sort_values(['PatientID','Days'],axis=1)
    return cpData

In [27]:
getPatientData(92,full_data).groupby('Days').nunique()
# need to group by days because of multiple visits per day. 

,VisitID,VisitName,VisitType,Age,Days,Diagnosis,DSMNo,Medication,Dose,CGI
Days,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1
4,2,2,1,1,1,2,2,1,1,1
55,1,1,1,1,1,1,1,1,1,1
56,1,1,1,1,1,1,1,1,1,1
99,1,1,1,1,1,1,1,1,1,1
102,2,1,1,1,1,2,2,1,1,1
103,3,1,1,1,1,2,2,1,1,2


In [37]:
import statistics
statistics.mean([1,2])

1.5

In [52]:
full_data.loc[full_data.PatientID == 2646]

,PatientID,VisitID,VisitName,VisitType,Age,Days,Diagnosis,DSMNo,Medication,Dose,CGI
3367,2646,61063,Evaluation,Discharge summary,25,27,Cocaine Abuse,305.60,clonazepam,2.0,4
3368,2646,61064,Progress Note,Inpatient,25,27,Opioid Abuse,305.50,clonazepam,2.0,4


In [51]:
@interact
def getCGIoverTime(pid=patient_list[100:110]):
    patient_data = getPatientData(pid, full_data)
    cgi = patient_data.groupby('Days').apply(lambda x: statistics.mean([x['CGI'].max(), x['CGI'].min()]))
    
    plt.plot(patient_data.groupby('Days')['CGI'].mean(), label='Severity', marker='x', color='blue')

    pid_daysmax = patient_data.Days.max()
    for yr in range(pid_daysmax//365): plt.axvline(x=(yr+1)*365,linestyle='--',c='grey')
    plt.axis([-10, pid_daysmax+10, -0.5,7.5])
    plt.title(f'CGI Severity of Patient {pid} over time')
    plt.show()
    return
    
getCGIoverTime()

interactive(children=(Dropdown(description='pid', options=(2598, 2637, 2646, 2713, 2718, 2758, 2765, 2815, 281…

ValueError: Lengths must match to compare

In [29]:
print(len(patient_list), 'is too many patients for the widgets to handle')

34895 is too many patients for the widgets to handle


In [32]:
data = full_data
@interact
def showPatientStats(pt=patient_list[:10], x=np.arange(0,365)):
    pt_data = data[data.PatientID==pt]
    return pt_data

interactive(children=(Dropdown(description='pt', options=(9, 19, 21, 89, 92, 93, 153, 299, 307, 316), value=9)…